### Setup AWS Credentials

In [ ]:
import os
# Setup your AWS Access Key and Secret Key as environment variables.
os.environ["AWS_ACCESS_KEY_ID"]
os.environ["AWS_SECRET_ACCESS_KEY"] 

In [1]:
# Setup Nova Model
NOVA_MODEL_ID = "us.amazon.nova-pro-v1:0"

### Dataset Adapter

Initialize the Dataset Adapter that takes the input_columns and output_columns. We use the JSONDatasetAdapter to read a `.jsonl` file and adapt it to the standardized format. We also use the adapter to create train and test sets for our use case.

In [2]:
from amzn_nova_prompt_optimizer.core.input_adapters.dataset_adapter import JSONDatasetAdapter

input_columns = {"input"}
output_columns = {"answer"}

dataset_adapter = JSONDatasetAdapter(input_columns, output_columns)

# Adapt
dataset_adapter.adapt("../data/FacilitySupportAnalyzer.jsonl")

train_set, test_set = dataset_adapter.split(0.5)

### Prompt Adapter

Initialize the Prompt Adapter for the Original Prompt. For this example, we use the FacilitySupportAnalyzer User Prompt in the `.txt` format. 

In [3]:
from amzn_nova_prompt_optimizer.core.input_adapters.prompt_adapter import TextPromptAdapter

prompt_variables = input_columns

prompt_adapter = TextPromptAdapter()

prompt_adapter.set_user_prompt(file_path="original_prompt/user_prompt_as_template.txt", variables=prompt_variables)

# Adapt
prompt_adapter.adapt()

2025/07/02 20:45:28 INFO amzn_nova_prompt_optimizer.core.input_adapters.prompt_adapter: System Prompt not set, initializing as empty string...


### Metric Adapter

Initialize the Metric Adapter for evaluating this prompt for certain optimizers. For this example, we build a Custom Metric for the FacilitySupportAnalyzer Dataset. The metric adapter requires the use of the `apply` [For single row evaluation] or `batch_apply` [For evaluating the whole dataset together] function

In [4]:
from amzn_nova_prompt_optimizer.core.input_adapters.metric_adapter import MetricAdapter
from typing import List, Any, Dict
import re
import json

class FacilitySupportAnalyzerMetric(MetricAdapter):
    def parse_json(self, input_string: str):
        """
        Attempts to parse the given string as JSON. If direct parsing fails,
        it tries to extract a JSON snippet from code blocks formatted as:
            ```json
            ... JSON content ...
            ```
        or any code block delimited by triple backticks and then parses that content.
        """
        try:
            return json.loads(input_string)
        except json.JSONDecodeError as err:
            error = err

        patterns = [
            re.compile(r"```json\s*(.*?)\s*```", re.DOTALL | re.IGNORECASE),
            re.compile(r"```(.*?)```", re.DOTALL)
        ]

        for pattern in patterns:
            match = pattern.search(input_string)
            if match:
                json_candidate = match.group(1).strip()
                try:
                    return json.loads(json_candidate)
                except json.JSONDecodeError:
                    continue

        raise error

    def _calculate_metrics(self, y_pred: Any, y_true: Any) -> Dict:
        strict_json = False
        result = {
            "is_valid_json": False,
            "correct_categories": 0.0,
            "correct_sentiment": False,
            "correct_urgency": False,
        }

        try:
            y_true = y_true if isinstance(y_true, dict) else (json.loads(y_true) if strict_json else self.parse_json(y_true))
            y_pred = y_pred if isinstance(y_pred, dict) else (json.loads(y_pred) if strict_json else self.parse_json(y_pred))
        except json.JSONDecodeError:
            result["total"] = 0
            return result  # Return result with is_valid_json = False
        else:
            if isinstance(y_pred, str):
                result["total"] = 0
                return result  # Return result with is_valid_json = False
            result["is_valid_json"] = True

            categories_true = y_true.get("categories", {})
            categories_pred = y_pred.get("categories", {})

            if isinstance(categories_true, dict) and isinstance(categories_pred, dict):
                correct = sum(
                    categories_true.get(k, False) == categories_pred.get(k, False)
                    for k in categories_true
                )
                result["correct_categories"] = correct / len(categories_true) if categories_true else 0.0
            else:
                result["correct_categories"] = 0.0  # or raise an error if you prefer

            result["correct_sentiment"] = y_pred.get("sentiment", "") == y_true.get("sentiment", "")
            result["correct_urgency"] = y_pred.get("urgency", "") == y_true.get("urgency", "")

        # Compute overall metric score
        result["total"] = sum(
            float(result[k]) for k in ["correct_categories", "correct_sentiment", "correct_urgency"]
        ) / 3.0

        return result

    def apply(self, y_pred: Any, y_true: Any):
        return self._calculate_metrics(y_pred, y_true)

    def batch_apply(self, y_preds: List[Any], y_trues: List[Any]):
        evals = [self.apply(y_pred, y_true) for y_pred, y_true in zip(y_preds, y_trues)]
        float_keys = [k for k, v in evals[0].items() if isinstance(v, (int, float, bool))]
        return {k: sum(e[k] for e in evals) / len(evals) for k in float_keys}

metric_adapter = FacilitySupportAnalyzerMetric()

### Inference Adapter
Initialize the InferenceAdapter to choose the backend Inference. Currently, we only support BedrockInferenceAdapter.

In [5]:
from amzn_nova_prompt_optimizer.core.inference.adapter import BedrockInferenceAdapter

inference_adapter = BedrockInferenceAdapter(region_name="us-east-1")

### Evaluator

The Evaluator can use the metric_adapter, prompt_adapter, and dataset_adapter to evaluate the prompt given the `model_id` to produce an evaluation score. The Evaluator internally uses the `InferenceRunner` to first generate inference results and then evaluate the output.

#### Base Model Evaluation

In [7]:
from amzn_nova_prompt_optimizer.core.evaluation import Evaluator

evaluator = Evaluator(prompt_adapter, test_set, metric_adapter, inference_adapter)

In [8]:
original_prompt_score = evaluator.aggregate_score(model_id=NOVA_MODEL_ID)

print(f"Original Prompt Evaluation Score = {original_prompt_score}")

2025/07/02 04:35:03 INFO amzn_nova_prompt_optimizer.core.evaluation: Cache miss - Running new inference on Dataset
Running inference: 100%|██████████████████████| 100/100 [00:45<00:00,  2.21it/s]
2025/07/02 04:35:48 INFO amzn_nova_prompt_optimizer.core.evaluation: Running Batch Evaluation on Dataset, using `batch_apply` metric
2025/07/02 04:35:48 INFO amzn_nova_prompt_optimizer.core.evaluation: Using cached inference results
2025/07/02 04:35:48 INFO amzn_nova_prompt_optimizer.core.evaluation: Running Evaluation on Dataset, using `apply` metric


Original Prompt Evaluation Score = {'is_valid_json': 0.27, 'correct_categories': 0.251, 'correct_sentiment': 0.13, 'correct_urgency': 0.2, 'total': 0.19366666666666668}


### Optimization Adapter

We can now define the Optimization Functions. The Optimization function takes as input the Prompt Adapter and Optionally a Dataset Adapter, Inference Adapter, and Metric Adapter. The optimization function optimizes the prompt and returns a Prompt Adapter.

In [6]:
class FacilitySupportAnalyzerNovaMetric(FacilitySupportAnalyzerMetric):
    def apply(self, y_pred: Any, y_true: Any):
        # Requires to return a value and not a JSON payload
        return self._calculate_metrics(y_pred, y_true)["total"]
        
    def batch_apply(self, y_preds: List[Any], y_trues: List[Any]):
        pass
nova_metric_adapter = FacilitySupportAnalyzerNovaMetric()

#### NovaPromptOptimizer

NovaPromptOptimizer = Nova Meta Prompter + MIPROv2 with Nova Model Tips

In [7]:
from amzn_nova_prompt_optimizer.core.optimizers import NovaPromptOptimizer

nova_prompt_optimizer = NovaPromptOptimizer(prompt_adapter=prompt_adapter, inference_adapter=inference_adapter, dataset_adapter=train_set, metric_adapter=nova_metric_adapter)

optimized_prompt_adapter = nova_prompt_optimizer.optimize(mode="pro")

2025/07/02 20:45:38 INFO amzn_nova_prompt_optimizer.core.optimizers.nova_meta_prompter.nova_mp_optimizer: Optimizing prompt using Nova Meta Prompter
2025/07/02 20:45:44 INFO amzn_nova_prompt_optimizer.core.optimizers.miprov2.miprov2_optimizer: Using us.amazon.nova-pro-v1:0 for Evaluation
2025/07/02 20:45:44 INFO amzn_nova_prompt_optimizer.core.optimizers.miprov2.miprov2_optimizer: Using us.amazon.nova-premier-v1:0 for Prompting
2025/07/02 20:45:44 INFO amzn_nova_prompt_optimizer.core.optimizers.miprov2.custom_adapters.custom_chat_adapter: Initializing CustomChatAdapter with enable_json_fallback=False
2025/07/02 20:45:44 INFO amzn_nova_prompt_optimizer.core.optimizers.miprov2.miprov2_optimizer: Using Nova tips for MIPROv2 optimization
2025/07/02 20:45:44 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/07/02 20:45:44 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instruc

Bootstrapping set 1/20
Bootstrapping set 2/20
Bootstrapping set 3/20


  8%|███████▋                                                                                        | 4/50 [00:06<01:09,  1.51s/it]


Bootstrapped 4 full traces after 4 examples for up to 1 rounds, amounting to 4 attempts.
Bootstrapping set 4/20


  8%|███████▋                                                                                        | 4/50 [00:05<01:04,  1.41s/it]


...

2025/07/02 21:01:34 INFO dspy.evaluate.evaluate: Average Metric: 27.066666666666666 / 35 (77.3%)
2025/07/02 21:01:34 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 77.33 on minibatch of size 35 with parameters ['Predictor 0: Instruction 17', 'Predictor 0: Few-Shot Set 1'].
2025/07/02 21:01:34 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [72.86, 71.9, 77.52, 72.57, 71.62, 74.1, 76.29, 69.43, 72.0, 75.81, 77.24, 76.86, 76.29, 79.24, 73.05, 78.86, 73.05, 79.14, 72.0, 72.86, 64.95, 75.14, 75.52, 70.29, 68.95, 75.05, 74.86, 73.9, 77.33]
2025/07/02 21:01:34 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [70.33, 77.07, 80.07, 77.13, 78.4, 78.6]
2025/07/02 21:01:34 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 80.07
2025/07/02 21:01:34 INFO dspy.teleprompt.mipro_optimizer_v2: ==========================================


2025/07/02 21:01:34 INFO dspy.teleprompt.mipro_optimizer_v2: == Trial 36 / 37 - Minibatch ==



Average Metric: 28.23 / 35 (80.7%): 100%|███████████████████████████████████████████████████████████| 35/35 [00:22<00:00,  1.53it/s]

2025/07/02 21:01:57 INFO dspy.evaluate.evaluate: Average Metric: 28.233333333333334 / 35 (80.7%)
2025/07/02 21:01:57 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 80.67 on minibatch of size 35 with parameters ['Predictor 0: Instruction 12', 'Predictor 0: Few-Shot Set 14'].
2025/07/02 21:01:57 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [72.86, 71.9, 77.52, 72.57, 71.62, 74.1, 76.29, 69.43, 72.0, 75.81, 77.24, 76.86, 76.29, 79.24, 73.05, 78.86, 73.05, 79.14, 72.0, 72.86, 64.95, 75.14, 75.52, 70.29, 68.95, 75.05, 74.86, 73.9, 77.33, 80.67]
2025/07/02 21:01:57 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [70.33, 77.07, 80.07, 77.13, 78.4, 78.6]
2025/07/02 21:01:57 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 80.07
2025/07/02 21:01:57 INFO dspy.teleprompt.mipro_optimizer_v2: ==========================================


2025/07/02 21:01:57 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 37 / 37 - Full Evaluation =====
202


Average Metric: 38.43 / 50 (76.9%): 100%|███████████████████████████████████████████████████████████| 50/50 [00:09<00:00,  5.19it/s]

2025/07/02 21:02:07 INFO dspy.evaluate.evaluate: Average Metric: 38.43333333333333 / 50 (76.9%)
2025/07/02 21:02:07 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [70.33, 77.07, 80.07, 77.13, 78.4, 78.6, 76.87]
2025/07/02 21:02:07 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 80.07
2025/07/02 21:02:07 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/07/02 21:02:07 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/07/02 21:02:07 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 80.07!


In [8]:
optimized_prompt_adapter.show()

2025/07/02 21:02:07 INFO amzn_nova_prompt_optimizer.core.input_adapters.prompt_adapter: 
Standardized Prompt:


{
  "user_prompt": {
    "variables": [
      "input"
    ],
    "template": "Extract and return a json with the following keys and values from the input provided: [{{ input }}]\n- \"urgency\" as one of `high`, `medium`, `low`\n- \"sentiment\" as one of `negative`, `neutral`, `positive`\n- \"categories\" as a dictionary with categories as keys and boolean values indicating if the category matches the input. Categories are: `emergency_repair_services`, `routine_maintenance_requests`, `quality_and_safety_concerns`, `specialized_cleaning_services`, `general_inquiries`, `sustainability_and_environmental_practices`, `training_and_support_requests`, `cleaning_services_scheduling`, `customer_feedback_and_complaints`, `facility_management_issues`",
    "metadata": {
      "format": "text"
    }
  },
  "system_prompt": {
    "variables": [],
    "template": "Analyze the input text and classify it into a JSON object with \"urgency\", \"sentiment\", and \"categories\" keys. Ensure all categories 

### Optimized System Prompt

In [9]:
print(optimized_prompt_adapter.system_prompt)

Analyze the input text and classify it into a JSON object with "urgency", "sentiment", and "categories" keys. Ensure all categories are covered with boolean values, and the output is a valid JSON string without extra formatting.


### Optimized User Prompt

In [10]:
print(optimized_prompt_adapter.user_prompt)

Extract and return a json with the following keys and values from the input provided: [{{ input }}]
- "urgency" as one of `high`, `medium`, `low`
- "sentiment" as one of `negative`, `neutral`, `positive`
- "categories" as a dictionary with categories as keys and boolean values indicating if the category matches the input. Categories are: `emergency_repair_services`, `routine_maintenance_requests`, `quality_and_safety_concerns`, `specialized_cleaning_services`, `general_inquiries`, `sustainability_and_environmental_practices`, `training_and_support_requests`, `cleaning_services_scheduling`, `customer_feedback_and_complaints`, `facility_management_issues`


### Few Shot Examples

In [11]:
print(f"Number of Few-Shot Examples = {len(optimized_prompt_adapter.few_shot_examples)}")

Number of Few-Shot Examples = 4


In [12]:
# Print only the first example
print(optimized_prompt_adapter.few_shot_examples[0])

{'input': 'Extract and return a json with the following keys and values from the input provided: [Subject: Request for Post-Renovation Cleaning Services\n\nDear ProCare Support Team,\n\nI hope this message finds you well. My name is [Sender], and I have been utilizing ProCare Facility Solutions for the maintenance and management of my commercial property for the past few years. Your services have always been reliable and efficient, which is why I am reaching out to you today.\n\nI am writing to request assistance with a specialized cleaning service for our office building. We have recently undergone some renovations, and there is a significant amount of dust and debris that needs to be addressed. Additionally, the carpets and windows require a thorough cleaning to restore them to their original condition.\n\nI have already scheduled a routine cleaning, but I believe this situation requires more specialized attention. Given the nature of the work needed, I would appreciate it if you cou

### Evaluator

Now we evaluate the Nova Prompt Optimizer Optimized prompt

In [13]:
from amzn_nova_prompt_optimizer.core.evaluation import Evaluator

evaluator = Evaluator(optimized_prompt_adapter, test_set, metric_adapter, inference_adapter)

In [14]:
evaluation_score = evaluator.aggregate_score(model_id=NOVA_MODEL_ID)
print(f"Nova Prompt Optimization = {evaluation_score}")

2025/07/02 21:02:07 INFO amzn_nova_prompt_optimizer.core.evaluation: Cache miss - Running new inference on Dataset
Running inference: 100%|██████████████████████████████████████████████████████████████████████████| 100/100 [00:44<00:00,  2.27it/s]
2025/07/02 21:02:51 INFO amzn_nova_prompt_optimizer.core.evaluation: Running Batch Evaluation on Dataset, using `batch_apply` metric
2025/07/02 21:02:51 INFO amzn_nova_prompt_optimizer.core.evaluation: Using cached inference results
2025/07/02 21:02:51 INFO amzn_nova_prompt_optimizer.core.evaluation: Running Evaluation on Dataset, using `apply` metric


Nova Prompt Optimization = {'is_valid_json': 1.0, 'correct_categories': 0.9309999999999999, 'correct_sentiment': 0.66, 'correct_urgency': 0.84, 'total': 0.8103333333333333}


In [15]:
optimized_prompt_adapter.save("optimized_prompt/")